# Analiza podatkov za projektno nalogo

Pozdravljeni,
v tem zvežčiču bom analiziral uroke, katere sem pobral iz spletne strani [dnd5e][1]. Uroki so sicer plačljivo gradivo, ki je raztrošeno po mnogih knjigah lastnika igre *Dungeons and Dragons*, ampak ker so hkrati dostopni tudi na tej brezplačni spletni strani, bom rekel, da je moja analiza nesporna. 

## Nekaj malega o igri

Najpomembnejši v igri so igralci z njihovimi junaki. Junak je oseba, kateri igralec določi raso, ceh, ozadje in tekom igre še veliko več. Kar se tiče izdelave junaka je igralcem dana precejšna svoboda, ampak za lažji potek in uravnovešenost igre se je pametno držati v naprej podanih okvirjev. Tako junaki pridobijo kompetence za spopadanje z nalogami, ki jim jih zadajo [NPC][2]-ji in zmožnost bojevanja. Tu imajo uroki pomembno vlogo, saj jih igralec lahko uporablja bodisi na vsakodnevni ravni za razreševanje problemov bodisi v boju za okrevanje in zadajanje škode. 

Pomembna lastnost uroka je tudi stopnja, ki mu je dodeljena. Na določeni stopnji junaka, so nam na voljo le uroki do vključno neke stopnje. Če to ponazorim: junak, ki je šele začel z igro bo lahko uporabljal le uroke na ničelni in prvi stopnji, proti koncu igre pa bo zmožen izvesti tudi uroke na deveti stopnji, ki so neprimerno močnejši.

Vsak urok ima osnovne karakteristike, ki so: 

- ime (`uroki.csv`), 
- šolo čaranja (`sola.csv`), 
- čas potreben za izvedbo uroka (`urocanje.csv`), 
- možnost izvedbe uroka v obliki rituala (`uroki.csv`),
- doseg (`doseg.csv`), 
- čas, ko je urok aktiven (`efekt.csv`),
- in za izvedbo potrebne komponente (materialne, somatske in verbalne) (`uroki.csv`),
- stopnjo uroka, ki je med 0 in 9 (`uroki.csv`).

Te podatke sem pobral s svetovnega spleta in jih uredil v `csv` datoteke. Poleg teh lastnosti imajo uroki tudi natančne opise, do katerih lahko dostopate preko `http://dnd5e.wikidot.com/spell:{urok_href}`, kjer je za vsak urok `urok_href` možno najti v datoteki `uroki.csv`.



[1]: <http://dnd5e.wikidot.com/spells> "Uroki v D&D"
[2]: <https://en.wikipedia.org/wiki/Non-player_character> "Več o NPC-jih"

In [10]:
import pandas as pd

In [11]:
uroki = pd.read_csv('podatki/uroki.csv')

In [12]:
uroki

,urok_href,urok,sola,urocanje,doseg,efekt,ritual,verbal,somatic,material,stopnja
0,acid-splash,Acid Splash,1,1,1,1,False,True,True,False,0
1,blade-ward,Blade Ward,2,1,2,2,False,True,True,False,0
2,booming-blade,Booming Blade,3,1,3,2,False,False,True,True,0
3,chill-touch,Chill Touch,4,1,4,2,False,True,True,False,0
4,control-flames,Control Flames,5,1,1,3,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...
556,time-stop,Time Stop,5,1,2,1,False,True,False,False,9
557,true-polymorph,True Polymorph,5,1,6,14,False,True,True,True,9
558,true-resurrection,True Resurrection,4,5,5,1,False,True,True,True,9
559,weird,Weird,8,1,4,4,False,True,True,False,9


In [13]:
sola = pd.read_csv('podatki/sola.csv')

In [14]:
urocanje = pd.read_csv('podatki/urocanje.csv')

In [15]:
doseg = pd.read_csv('podatki/doseg.csv')

In [16]:
efekt = pd.read_csv('podatki/efekt.csv')

## Vsakodnevna uporaba

V igri je vsak dan malo drugačen, a po svoje vedno pester, zato se vedno izvaja neka coprnija. Ampak! Tu nas čaka precejšen problem, saj je število urokov, ki jih lahko izvedemo na dan zelo omejeno. Imamo pa vseeno možnost izvedbe uroka v ritualni obliki. To nam omogoči dobiti vse prednosti uroka na osnovni ravni, če ga uročamo 10 minut dlje. Žal pa možnost ritualnega uročanja ni dana vsakemu uroku, zato nas zanima koliko ritualnih urokov obstaja za vsako stopnjo, ter kolikšen je delež le-teh. S temi informacijami se bomo dokopali do dejanskega arsenala urokov, ki ga imamo vsak dan.

Tu bi še dodal, da mnogo pravil, ki jih omenjam, velja le za ceh *Cleric*, saj ta na začetku vsakega dne izbere uroke, ki so mu na voljo tisti dan pri dani junakovi stopnji. Pri preostalih cehih izgleda pridobivanje zelo drugačno.

In [22]:
ritualni = uroki[uroki.ritual]

In [23]:
ritualni

,urok_href,urok,sola,urocanje,doseg,efekt,ritual,verbal,somatic,material,stopnja
52,alarm,Alarm,2,1,6,6,True,True,True,True,1
63,ceremony,Ceremony,3,1,5,1,True,True,True,True,1
70,comprehend-languages,Comprehend Languages,7,1,2,7,True,True,True,True,1
74,detect-magic,Detect Magic,7,1,2,15,True,True,True,False,1
87,find-familiar,Find Familiar,1,5,9,1,True,True,True,True,1
94,guiding-hand-ua,Guiding Hand (UA),7,3,7,16,True,True,True,False,1
104,identify,Identify,7,3,5,1,True,True,True,True,1
116,purify-food-and-drink,Purify Food and Drink,5,1,9,1,True,True,True,False,1
127,speak-with-animals,Speak with Animals,7,1,2,10,True,True,True,False,1
131,tensers-floating-disk,Tenser's Floating Disk,1,1,6,7,True,True,True,True,1


In [29]:
a = ritualni.groupby('stopnja').size()

In [30]:
a

stopnja
1    12
2     9
3     6
4     2
5     5
6     2
dtype: int64